In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error


In [2]:
df = pd.read_csv("/Users/daniel.al-choboq/Documents/Code/Missions/axa/data/pg15training.csv")
df.head()

# export DYLD_LIBRARY_PATH=/opt/homebrew/opt/libomp/lib:$DYLD_LIBRARY_PATH

,PolNum,CalYear,Gender,Type,Category,Occupation,Age,Group1,Bonus,Poldur,Value,Adind,SubGroup2,Group2,Density,Exppdays,Numtppd,Numtpbi,Indtppd,Indtpbi
0,200114978,2009,Male,C,Large,Employed,25,18,90,3,15080,0,L46,L,72.012883,365,1,0,0.0,0.0
1,200114994,2009,Male,E,Large,Employed,20,11,30,2,22370,1,O38,O,39.550411,365,1,0,0.0,0.0
2,200115001,2009,Female,E,Large,Unemployed,42,11,150,0,39650,0,Q28,Q,169.529148,365,2,0,0.0,0.0
3,200115011,2009,Female,C,Medium,Housewife,21,5,0,0,12600,1,L6,L,58.894688,365,1,0,0.0,0.0
4,200115015,2009,Female,D,Large,Employed,33,12,30,10,9065,0,N4,N,109.631885,365,2,0,0.0,0.0


In [3]:
df["Numtppd"].value_counts()

Numtppd
0    87744
1    10358
2     1518
3      306
4       62
5       17
6        9
7        7
Name: count, dtype: int64

In [4]:
df['target'] = df['Numtppd'].apply(lambda x: 1 if x != 0 else 0)


In [5]:
df['target'].value_counts()

target
0    87744
1    12277
Name: count, dtype: int64

In [7]:
# Convert object columns to categorical
categorical_columns = ['Gender', 'Type', 'Category', 'Occupation', 'SubGroup2', 'Group2']
for col in categorical_columns:
    df[col] = df[col].astype('category')

# Use LightGBM instead of XGBoost
# Add one hot encoder with pipeline scikit learn

# Define features and target
X = df.drop(columns=['Numtppd', 'Numtpbi', 'Indtppd', 'Indtpbi', 'target'])
y = df['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the LightGBM model
import lightgbm as lgb
model = lgb.LGBMClassifier(objective='binary', n_estimators=100, learning_rate=0.1, max_depth=5)

# Train the model
model.fit(X_train, y_train, categorical_feature=categorical_columns)

# Make predictions
y_pred = model.predict(X_test)
 
# Evaluate the model, add more metrics
accuracy = (y_pred == y_test).mean()
print(f"Accuracy: {accuracy}")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 9713, number of negative: 70303
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000502 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1596
[LightGBM] [Info] Number of data points in the train set: 80016, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.121388 -> initscore=-1.979349
[LightGBM] [Info] Start training from score -1.979349
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes